# Shallow Water Tutorial 1D with Numpy (Simple)


::: {.callout-note title="Reference"}
The following the model is described in the paper: 

```
 @article{Delestre_2013, 
 title={SWASHES: a compilation of Shallow Water Analytic Solutions for Hydraulic and Environmental Studies}, 
 volume={72}, 
 ISSN={0271-2091, 1097-0363}, DOI={10.1002/fld.3741}, 
 note={arXiv:1110.0288 [physics]}, 
 number={3}, 
 journal={International Journal for Numerical Methods in Fluids}, 
 author={Delestre, Olivier and Lucas, Carine and Ksinant, Pierre-Antoine and Darboux, Frédéric and Laguerre, Christian and Vo, Thi Ngoc Tuoi and James, Francois and Cordier, Stephane}, 
 year={2013}, 
 month=may, 
 pages={269–300} 
}
```

:::

## Imports

In [1]:
import sys

if "pyodide" in sys.modules:
    import micropip

    await micropip.install("zoomy-core")

In [3]:
import zoomy_core
zoomy_core.__version__

'0.1.13'

In [6]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np


from sympy import Matrix
import pytest

from zoomy_core.fvm.solver_numpy import HyperbolicSolver, Settings
import zoomy_core.fvm.timestepping as timestepping
import zoomy_core.fvm.nonconservative_flux as nc_flux
from zoomy_core.model.basemodel import Model
from attr import field, define
import zoomy_core.model.initial_conditions as IC
import zoomy_core.model.boundary_conditions as BC
import zoomy_core.misc.io as io
from zoomy_core.mesh.mesh import compute_derivatives
from zoomy_core.misc.misc import Zstruct, ZArray



import zoomy_core.mesh.mesh as petscMesh
import zoomy_core.postprocessing.postprocessing as postprocessing


In [7]:
@define(frozen=True, slots=True, kw_only=True)
class SWE(Model):
    dimension: int = 1
    variables: Zstruct = field(init=False, default=dimension + 1)
    aux_variables: Zstruct = field(default=1)
    _default_parameters: dict = field(
        init=False,
        factory=lambda: {"g": 9.81, "ex": 0.0, "ey": 0.0, "ez": 1.0}
        )


    def project_2d_to_3d(self):
        out = ZArray.zeros(6)
        dim = self.dimension
        z = self.position[2]
        b = 0
        h = self.variables[0]
        U = [hu / h for hu in self.variables[1:1+dim]]
        rho_w = 1000.
        g = 9.81
        out[0] = b
        out[1] = h
        out[2] = U[0]
        out[3] = 0 if dim == 1 else U[1]
        out[4] = 0
        out[5] = rho_w * g * h * (1-z)
        return out

    def flux(self):
        dim = self.dimension
        h = self.variables[0]
        U = Matrix([hu / h for hu in self.variables[1:1+dim]])
        g = self.parameters.g
        I = Matrix.eye(dim)
        F = Matrix.zeros(self.variables.length(), dim)
        F[0, :] = (h * U)
        F[1:, :] = h * U * U.T + g/2 * h**2 * I
        
        return F

In [8]:

bcs = BC.BoundaryConditions(
    [
        BC.Extrapolation(tag="left"),
        BC.Extrapolation(tag="right"),
    ]
)

def custom_ic(x):
    Q = np.zeros(2, dtype=float)
    Q[0] = np.where(x[0] < 5., 0.005, 0.001)
    return Q

ic = IC.UserFunction(custom_ic)

model = SWE(
    boundary_conditions=bcs,
    initial_conditions=ic,
)

main_dir = os.getenv("ZOOMY_DIR")
# mesh = petscMesh.Mesh.from_gmsh(
#     os.path.join(main_dir, "meshes/channel_quad_2d/mesh.msh")
# )

mesh = petscMesh.Mesh.create_1d(domain=(0.0, 10.0), n_inner_cells=500)

class SWESolver(HyperbolicSolver):
    def update_qaux(self, Q, Qaux, Qold, Qauxold, mesh, model, parameters, time, dt):
        dudx = compute_derivatives(Q[1]/Q[0], mesh, derivatives_multi_index=[[0, 0]])[:,0]
        Qaux[0]=dudx
        return Qaux
    
settings = Settings(name="ShallowWater", output=Zstruct(directory="outputs/shallow_water_1d", filename="swe", clean_directory=True))


In [9]:
solver = SWESolver(time_end=6.0, settings=settings, compute_dt=timestepping.adaptive(CFL=0.95))
Qnew, Qaux = solver.solve(mesh, model)

2025-11-07 16:22:50.726 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 10, time: 0.718260, dt: 0.067958, next write at time: 0.666667
2025-11-07 16:22:50.824 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 20, time: 1.392979, dt: 0.067263, next write at time: 1.333333
2025-11-07 16:22:50.923 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 30, time: 2.064282, dt: 0.067054, next write at time: 2.000000
2025-11-07 16:22:51.014 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 40, time: 2.734237, dt: 0.066956, next write at time: 3.333333
2025-11-07 16:22:51.112 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 50, time: 3.403455, dt: 0.066898, next write at time: 4.000000
2025-11-07 16:22:51.214 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 60, time: 4.072203, dt: 0.066858, next write at time: 4.666667
2025-11-07 16:22:51.303 | INFO     | zoomy_core.fvm.solver_numpy:run:285 - iteration: 70, time: 4.740621, 

In [10]:
io.generate_vtk(os.path.join(settings.output.directory, f"{settings.output.filename}.h5"))
postprocessing.vtk_project_2d_to_3d(model, settings, filename='out_3d')

2025-11-07 16:22:53.570 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 0/10
2025-11-07 16:22:53.921 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 1/10
2025-11-07 16:22:54.271 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 2/10
2025-11-07 16:22:54.616 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 3/10
2025-11-07 16:22:55.063 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 4/10
2025-11-07 16:22:55.445 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 5/10
2025-11-07 16:22:55.918 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - Converted snapshot 6/10
2025-11-07 16:22:56.381 | INFO     | zoomy_core.postprocessing.postprocessing:vtk_project_2d_to_3d:71 - 

In [11]:
# fig = plots_paper.plot_swe(os.path.join(settings.output.directory, settings.output.filename + ".h5"))

<class 'NameError'>: name 'plots_paper' is not defined

In [17]:
path = os.path.join(settings.output.directory, f"{settings.output.filename}.h5")
mesh = io.load_mesh_from_hdf5(path)
data = io.load_fields_from_hdf5(path)

In [18]:
X = mesh.cell_centers[0]
Y = data

array([ 1.000e-02,  3.000e-02,  5.000e-02,  7.000e-02,  9.000e-02,
        1.100e-01,  1.300e-01,  1.500e-01,  1.700e-01,  1.900e-01,
        2.100e-01,  2.300e-01,  2.500e-01,  2.700e-01,  2.900e-01,
        3.100e-01,  3.300e-01,  3.500e-01,  3.700e-01,  3.900e-01,
        4.100e-01,  4.300e-01,  4.500e-01,  4.700e-01,  4.900e-01,
        5.100e-01,  5.300e-01,  5.500e-01,  5.700e-01,  5.900e-01,
        6.100e-01,  6.300e-01,  6.500e-01,  6.700e-01,  6.900e-01,
        7.100e-01,  7.300e-01,  7.500e-01,  7.700e-01,  7.900e-01,
        8.100e-01,  8.300e-01,  8.500e-01,  8.700e-01,  8.900e-01,
        9.100e-01,  9.300e-01,  9.500e-01,  9.700e-01,  9.900e-01,
        1.010e+00,  1.030e+00,  1.050e+00,  1.070e+00,  1.090e+00,
        1.110e+00,  1.130e+00,  1.150e+00,  1.170e+00,  1.190e+00,
        1.210e+00,  1.230e+00,  1.250e+00,  1.270e+00,  1.290e+00,
        1.310e+00,  1.330e+00,  1.350e+00,  1.370e+00,  1.390e+00,
        1.410e+00,  1.430e+00,  1.450e+00,  1.470e+00,  1.490e

In [ ]:
@pytest.mark.nbworking
def test_working():
    assert True